In [1]:
import scipy.io
import numpy as np
import datetime 
import ephem
import random
from scipy import spatial
import matplotlib.pyplot as plt
%matplotlib
from astropy.table import Table


data=scipy.io.loadmat('BNS-sources-trial-10saleem.mat')['data'] 
#sourceparameters
DL=data[:,0]
theta=data[:,1]
phi=data[:,2]

Using matplotlib backend: TkAgg


In [2]:
#snr at each detector
snrH=data[:,5]
snrL=data[:,6]
snrV=data[:,7]
snrK=data[:,8]
snrI=data[:,9]
#or for networks of 5 and 3 detectors, one can use directly,
snrLHVKI=np.sqrt(data[:,5]**2 + data[:,6]**2 + data[:,7]**2 +data[:,8]**2 + data[:,9]**2)  
# SNR at LHVKI
snrLHV=np.sqrt(data[:,5]**2 + data[:,6]**2 + data[:,7]**2 )  # LHV*
snrLHVK = np.sqrt(data[:,5]**2 + data[:,6]**2 + data[:,7]**2 + data[:,8]**2)
snrLHVI = np.sqrt(data[:,5]**2 + data[:,6]**2 + data[:,7]**2 + data[:,9]**2)
snrH, snrL, snrLHV,snrLHVK, snrLHVI

(array([1.50828927, 1.3218442 , 4.88369119, ..., 0.14711676, 1.92059118,
        3.73704228]),
 array([2.05597716, 0.7623997 , 3.94531224, ..., 0.87258219, 1.87887723,
        3.17437839]),
 array([2.60109925, 2.01527692, 6.56372816, ..., 2.09722166, 3.03471228,
        5.1190831 ]),
 array([2.9615534 , 2.05214273, 7.23216427, ..., 2.10021926, 3.04157166,
        5.57032074]),
 array([2.80024108, 2.33763304, 7.55768521, ..., 2.6656951 , 3.20978535,
        5.91029335]))

In [3]:
from astropy.time import Time
#import random

def homotime(number):
    """
    Defines homogenously distributed time over the time 2022-2023. Number of time elements is essentially the number of
    theta,  phi you get.(220000)
    Takes time in UTC format and gives out time in UTC format
    """
    time1 = Time('2022-01-01T00:00:00.123456789')
    time1 = time1.jd
    time2 = Time('2023-01-01T00:00:00.123456789')
    time2 = time2.jd
    dt = (time2-time1)/number
    homotime = []
    
    for i in range(number):
        if i%(int((number)/10))==0:print(int(i*100/(number)), '% completed', end='\r')
        timejd = time1 + i*dt
        timeutc = Time(timejd, format='jd')
        homotime.append(timeutc.iso)
    print('***Completed***', end='\r')
    #print(len(homotime))
    return homotime
homotime = homotime(len(theta))


In [5]:
def ecef_eci(lat, lon, time):
    """Convert lat long to right ascention and declination
    lat - latitude of the place where the tigger is at the zenith in degrees
    lon - longitude of the place where the tigger is at the zenith in degrees
    time - UTC time in datetime format
    """
    lat = np.deg2rad(lat)
    lon = np.deg2rad(lon)
    temp = time
    year = int(temp[0:4])
    month = int(temp[5:7])
    day = int(temp[8:10])
    hour = int(temp[11:13])
    minu = int(temp[14:16])
    secs = int(temp[17:19])
    #time = Time(time)
    time = datetime.datetime(year, month,day,hour,minu,secs)
    time0 = datetime.datetime(2000, 1, 1, 12, 0, 0)
    time_sid = 18.697374558 + 24.06570982441908*(time - time0).total_seconds()/86400.0
    time_sid = time_sid - 24.0*np.floor(time_sid/24.0)
    dec = lat
    ra = lon + np.pi/24.0*time_sid
    dec = np.rad2deg(dec)
    ra = np.rad2deg(ra)
    return ra,dec


def eci_ecef(ra, dec, time):
    """Convert ra dec to lattitude and longitude
    ra - right ascention of the body in radians
    dec - declination of the body in radians
    time - UTC time in datetime format
    """
#     ra = np.deg2rad(ra)
#     dec = np.deg2rad(dec)
    temp = time
    year = int(temp[0:4])
    month = int(temp[5:7])
    day = int(temp[8:10])
    hour = int(temp[11:13])
    minu = int(temp[14:16])
    secs = int(temp[17:19])
    #time = Time(time)
    time = datetime.datetime(year, month,day,hour,minu,secs)
    time0 = datetime.datetime(2000, 1, 1, 12, 0, 0)
    time_sid = 18.697374558 + 24.06570982441908*(time - time0).total_seconds()/86400.0
    time_sid = time_sid - 24.0*np.floor(time_sid/24.0)
    lon = (ra - 2*np.pi/24.0*time_sid)
    lat = dec
#     lon = np.rad2deg(lon)
#     lat = np.rad2deg(lat)
    return lat, lon

def is_saa(lat, longi, time):
    """Return whether the satellite is in South Atlantic Anamoly or not.
    Hard bounds used from the data of RXTE and assumed that the SAA region is rectangular below the inclination of RXTE.
    
    ra - right ascention of the body in degrees
    dec - declination of the body in degrees
    time - UTC time in datetime format
    """
    # lat, longi = radec2latlong(ra, dec, time)
    #lat, longi = eci_ecef(ra, dec, time)
    lat = np.rad2deg(lat)
    longi = np.rad2deg(longi)
    if -23< lat <= -7 and -90<=longi<=-10:
        return True
    
#     if -90<=longi<=-10:
#         y = 7*longi/80 + 23/8
#         if lat < y:
#             return True
    if -100<=longi<=-35:
        y=14*longi/65+189/13
        if lat<=y:
            return True
    
    if -35<longi<-10:
        y=-14/25*longi-63/5
        if lat<y:
            return True
    
    
    if -10<=longi<=40:
        z = -9*longi/25 -43/5
        if lat<z: 
            return True
	

    if -50<= lat <=-23 and -90<=longi<=40:
        return True
    else: return False 

def getxyz(theta,phi,n):
    if n==1:
        #lat, long
        theta = np.pi/2 - theta
    x=np.sin(theta)*np.cos(phi)
    y=np.sin(theta)*np.sin(phi)
    z=np.cos(theta)
    return x,y,z
def getnorm(vec):
    norm = np.linalg.norm(vec)
    #print (norm)
    return vec/norm

In [6]:
line0 = 'ASTROSAT' #Daksha
line1 = '1 40930U 15052A   22302.17850942  .00000772  00000-0  19439-4 0  9990'
line2 = '2 40930   5.9975   0.5159 0009292 339.1937  20.7998 14.76165238112787'
line3 = '2 40930   5.9975   0.5159 0009292 339.1937 200.7998 14.76165238112787'

# line0 = 'SWIFT'#BAT
# line1 = '1 28485U 04047A   22302.18405080  .00001301  00000-0  60140-4 0  9997'
# line2 = '2 28485  20.5572 277.8742 0011565 130.4507 229.6936 15.03911323708409'
# line0 = 'FGRST (GLAST)'#Fermi
# line1 = '1 33053U 08029A   22100.45970249  .00000511  00000-0  12538-4 0  9998'
# line2 = '2 33053  25.5819 140.4060 0012690 120.1881 239.9912 15.11069703542313'
astrosat1 = ephem.readtle(line0,line1,line2)
astrosat2 = ephem.readtle(line0,line1,line3)

sublat1 = []
sublong1 = []
sublat2 = []
sublong2 = []
for i in range(len(homotime)):
    astrosat1.compute(homotime[i])
    astrosat2.compute(homotime[i])
    sublat1.append(astrosat1.sublat)#sublat in radians
    sublong1.append(astrosat1.sublong)#sublong in radian
    sublat2.append(astrosat2.sublat)#sublat in radians
    sublong2.append(astrosat2.sublong)#sublong in radian
    


In [7]:
sublat1[123], sublat2[123]
np.rad2deg(sublong1[123]), np.rad2deg(sublong2[123])

(-31.305171438040038, 148.4712097623955)

In [230]:
def randomlumino(number):
    """
    Returns 'number' of Luminosity values between the given caps
    """
    lumino = []
    lummax = -6
    lummin = -7
    factor = 10**6
    for i in range(number):
        #case1
        lumino.append(10**(random.randint(lummin*factor,lummax*factor)/factor))
        lumino[i] = lumino[i]*(200**2)/((DL[i])**2)
#         #case2
#         lumino.append(10**lummax)
#         lumino[i] = lumino[i]*(200**2)/((DL[i])**2)
#         #case3
#         if DL[i]<=200:
#             lumino.append(10**(random.randint(lummin*factor,lummax*factor)/factor))
#             lumino[i] = lumino[i]*(200**2)/((DL[i])**2)
#         else:
#             lumino.append(0.0)
#             lumino[i] = 0
#         #case4
#         if DL[i]<=200:
#             lumino.append(10**(lummin))
#             lumino[i] = lumino[i]*(200**2)/((DL[i])**2)
#         else:
#             lumino.append(0.0)
#             lumino[i] = 0
    return lumino
    
#         if DL[i]<=200:
#             lumino.append(10**(random.randint(lummin*factor,lummax*factor)/factor))
# #             lumino.append(10**(lummin))
#             lumino[i] = lumino[i]*(200**2)/((DL[i])**2)
#         else:
#             lumino.append(0.0)
# #             lumino[i] = 0
# #         lumino[i] = lumino[i]*(200**2)/((DL[i])**2)
# #         lumino.append(10**lummax)
        
#     return lumino

lumino = randomlumino(len(homotime))
# len(homotime)
# print(lumino)

def is_visible(time,latsat,longsat, lattrig, longtrig, satephem):
    satephem.compute(time)
    theta = 2*np.arcsin(6400000/(6400000+satephem.elevation))
    xsat = np.cos(longsat)*np.sin(np.pi/2 - latsat)
    ysat = np.sin(longsat)*np.sin(np.pi/2 - latsat)
    zsat = np.cos(np.pi/2 - latsat)
    satvec = (xsat,ysat,zsat)
    xtrig = np.cos(longtrig)*np.sin(np.pi/2 - lattrig)
    ytrig = np.sin(longtrig)*np.sin(np.pi/2 - lattrig)
    ztrig = np.cos(np.pi/2 - lattrig)
    trigvec = (xtrig,ytrig,ztrig)
    
    anglebetween = np.arccos(np.dot(satvec,trigvec))
    omegatrig = 2*np.pi*(1- np.cos(np.pi-anglebetween))
    omegashadow = 2*np.pi*(1- np.cos((theta/2)))
#     omegashadow = 2*np.pi*(1-np.cos(np.deg2rad(180-38.7394)))
    if omegashadow>omegatrig:
        return False
    else:
        return True
# plt.hist(np.log10(lumino), bins=100)
print(lumino)
plt.hist(lumino, bins=500)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


(array([2.5900e+04, 2.9002e+04, 2.3294e+04, 1.8989e+04, 1.6139e+04,
        1.3990e+04, 1.2127e+04, 1.1009e+04, 9.7200e+03, 8.0720e+03,
        6.4260e+03, 5.2250e+03, 4.3800e+03, 3.6170e+03, 3.0520e+03,
        2.5980e+03, 2.2480e+03, 1.9500e+03, 1.7930e+03, 1.4720e+03,
        1.3220e+03, 1.1870e+03, 1.0580e+03, 9.3900e+02, 8.3600e+02,
        7.7900e+02, 7.6000e+02, 6.5800e+02, 6.0100e+02, 5.6500e+02,
        4.9900e+02, 4.8300e+02, 4.4100e+02, 4.0800e+02, 3.6500e+02,
        3.6500e+02, 3.1500e+02, 2.8500e+02, 2.7100e+02, 2.8800e+02,
        2.5400e+02, 2.4500e+02, 2.1000e+02, 1.8700e+02, 2.1100e+02,
        1.9900e+02, 1.6800e+02, 1.6200e+02, 1.5800e+02, 1.4900e+02,
        1.5500e+02, 1.5100e+02, 1.3600e+02, 1.1100e+02, 1.3100e+02,
        1.1400e+02, 1.0500e+02, 1.0000e+02, 8.3000e+01, 1.0100e+02,
        1.0100e+02, 9.3000e+01, 9.2000e+01, 9.2000e+01, 9.2000e+01,
        8.4000e+01, 6.0000e+01, 6.9000e+01, 7.9000e+01, 6.4000e+01,
        5.8000e+01, 5.9000e+01, 6.0000e+01, 5.70

In [11]:
sensfile = np.loadtxt('sensitivity_20det.csv', delimiter=',',skiprows=1)

In [12]:

def transform(astrosat):
    thetatransarray = []
    phitransarray = []
    for i in range(len(theta)):
        if i%10000==0:
            print(i, end=' ')
        astrosat.compute(homotime[i])
        satvecold = getnorm(getxyz(astrosat.sublat,astrosat.sublong,1))
        tplus = homotime[i][0:17]+(str(int(homotime[i][17:19])+1)) + homotime[i][19:23]
        astrosat.compute(tplus)
        satvecplus = getnorm(getxyz(astrosat.sublat,astrosat.sublong,1))

        zaxis = getnorm(satvecold)
        xaxistemp = getnorm([satvecplus[0]-satvecold[0],satvecplus[1]-satvecold[1],satvecplus[2]-satvecold[2]])
        yaxis = getnorm(np.cross(zaxis,xaxistemp))
        xaxis = getnorm(np.cross(yaxis,zaxis))

        xtrig,ytrig,ztrig = getxyz(theta[i],phi[i],0)
        trigvec = getnorm([xtrig,ytrig,ztrig])

        thetatrans = np.arccos(np.dot(trigvec,zaxis))
        yprojtrig = np.linalg.norm(trigvec)*np.dot(trigvec,yaxis)
        xprojtrig = np.linalg.norm(trigvec)*np.dot(trigvec,xaxis)
        phitrans = np.arctan2(yprojtrig, xprojtrig)
        thetatransarray.append(thetatrans)
        phitransarray.append(phitrans)
    return thetatransarray,phitransarray

theta1,phi1 = transform(astrosat1)
theta2,phi2 = transform(astrosat2)


0 10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 110000 120000 130000 140000 150000 160000 170000 180000 190000 200000 210000 0 10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 110000 120000 130000 140000 150000 160000 170000 180000 190000 200000 210000 

In [13]:
sensitivitydata1 = []
sensitivitydata2 = []
sensfilethetaphi = []
for i in range(len(sensfile)):
    sensfilethetaphi.append((sensfile[i][0], sensfile[i][1]))
    

In [14]:
for j in range(len(theta)):
    if j%100==0:
        print(int(j*100/len(theta)),'% Completed', end='\r')
    vectrig = [theta1[j],phi1[j]]
    nearest = sensfilethetaphi[spatial.KDTree(sensfilethetaphi).query(vectrig)[1]]
    for i in range(len(sensfilethetaphi)):
        if sensfilethetaphi[i][0] == nearest[0]:
            if sensfilethetaphi[i][1]==nearest[1]:
                index=i
    
    sensitivitydata1.append(sensfile[index][3])

for j in range(len(theta)):
    if j%100==0:
        print(int(j*100/len(theta)),'% Completed', end='\r')
    vectrig = [theta2[j],phi2[j]]
    nearest = sensfilethetaphi[spatial.KDTree(sensfilethetaphi).query(vectrig)[1]]
    for i in range(len(sensfilethetaphi)):
        if sensfilethetaphi[i][0] == nearest[0]:
            if sensfilethetaphi[i][1]==nearest[1]:
                index=i
    
    sensitivitydata2.append(sensfile[index][3])

In [15]:
for k in range(len(sensitivitydata2)):
    if np.isnan(sensitivitydata2[k])==True:
        sensitivitydata2[k] = 0
        
sensdatabackup2 = sensitivitydata2

for k in range(len(sensitivitydata1)):
    if np.isnan(sensitivitydata1[k])==True:
        sensitivitydata1[k] = 0
        
sensdatabackup1 = sensitivitydata1

In [211]:
numtrig = 0
for i in range(len(lumino)):
    if lumino[i]>0:numtrig = numtrig + 1

print(numtrig)
# senssattemp = 1.4*10**(-7)
# senssattemp = 7.0*10**(-8)
boolvis = []
countsaa1, countsens1, countearth1, countvis1= 0,0,0,0
countsaa2, countsens2, countearth2, countvis2 = 0,0,0,0
count1sat,count1sat = 0,0
count2sat,count2sat = 0,0
count0sat,count0sat = 0,0
lowsnr = 0
for i in range(len(homotime)):
    if i%10000==0:
        print(int(i*100/len(homotime)),'%',end='\r', flush=True)
    if snrLHVKI[i]<snr:
            lowsnr=lowsnr + 1
    if lumino[i]>0:
        satyes, satyes = 0,0
        if is_saa(sublat1[i],sublong1[i], homotime[i])==True:
            countsaa1=countsaa1 + 1
        if sensitivitydata1[i]>lumino[i]:
            countsens1=countsens1+1
        if is_visible(homotime[i],sublat1[i],sublong1[i],np.deg2rad(90.0) - theta1[i],phi1[i], astrosat1)==False:
            countearth1 = countearth1 +1
        if is_saa(sublat1[i],sublong1[i], homotime[i])==False and snrLHVKI[i]>=8 and sensitivitydata1[i]<=lumino[i] and is_visible(homotime[i],sublat1[i],sublong1[i],np.deg2rad(90.0) - theta1[i],phi1[i], astrosat1)==True:
            countvis1 = countvis1+1
            satyes = satyes + 1

        if is_saa(sublat2[i],sublong2[i], homotime[i])==True:
            countsaa2=countsaa2 + 1
        if sensitivitydata2[i]>lumino[i]:
            countsens2=countsens2+1
        if is_visible(homotime[i],sublat2[i],sublong2[i],np.deg2rad(90.0) - theta2[i],phi2[i], astrosat1)==False:
            countearth2 = countearth2 +1
        if is_saa(sublat2[i],sublong2[i], homotime[i])==False and snrLHVKI[i]>=8 and sensitivitydata2[i]<=lumino[i] and is_visible(homotime[i],sublat2[i],sublong2[i],np.deg2rad(90.0) - theta2[i],phi2[i], astrosat2)==True:
            countvis2 = countvis2+1
            satyes = satyes + 1

        if satyes==0:
            count0sat=count0sat+1
        if satyes==1:
            count1sat=count1sat+1
        if satyes==2:
            count2sat=count2sat+1

print('***100% Completed***\n',end='\r',flush=True)         
print('Low SNR                                         ', lowsnr*100/numtrig)
print('Number of events visible                        ', countvis1*100/numtrig)
print('Blocked because of SAA                          ', countsaa1*100/numtrig)
print('Too faint                                       ', countsens1*100/numtrig)
print('Blocked because of earth occultation            ', countearth1*100/numtrig)
print('Number of events visible                        ', countvis2*100/numtrig)
print('Blocked because of SAA                          ', countsaa2*100/numtrig)
print('Too faint                                       ', countsens2*100/numtrig)
print('Blocked because of earth occultation            ', countearth2*100/numtrig)
print('0 satellites                                    ', count0sat*100/numtrig)
print('Either of the two satellite(seen by only one)   ', count1sat*100/numtrig)
print('2 satellites                                    ', count2sat*100/numtrig)
print((count1sat+count2sat+count0sat)*100/numtrig)
print(count1sat*100/numtrig+count2sat*100/numtrig)

print('lowsnr', lowsnr,'\t', lowsnr*100/numtrig)
print('goodsnr', numtrig - lowsnr,'\t', (numtrig - lowsnr)*100/numtrig)
print('countvis1', countvis1,'\t', countvis1*100/(numtrig-lowsnr))
print('countvis2', countvis2,'\t', countvis2*100/(numtrig-lowsnr))
#     print('At least one daksha', count1sat, '\t', count1sat*100/(numtrig-lowsnr))

print('Output')
print(countvis1, '\t', countvis1*100/(numtrig-lowsnr), '('+str(countvis1*100/numtrig)+ ')')
print(countvis2, '\t', countvis2*100/(numtrig-lowsnr),'('+ str(countvis2*100/numtrig)+ ')')
print((count1sat+count2sat), '\t',(count1sat+count2sat)*100/(numtrig-lowsnr) ,'(' + str(count1sat*100/numtrig+count2sat*100/numtrig)+ ')')



# print('\nNumber of events visible                        ', countvis*100/numtrig)
# print('Blocked because of SAA                          ', countsaa*100/numtrig)
# print('Too faint                                       ', countsens*100/numtrig)
# print('Blocked because of earth occultation            ', countearth*100/numtrig)

220000


KeyboardInterrupt: 

In [156]:
p1=str(countvis1)+'\t'+str(np.round(countvis1*100/(numtrig-lowsnr),2))+' ('+str(np.round(countvis1*100/numtrig,2))+')'
p2=str(countvis2)+'\t'+str(np.round(countvis2*100/(numtrig-lowsnr),2))+' ('+str(np.round(countvis2*100/numtrig,2))+')'
p3=str(count1sat+count2sat)+'\t'+str(np.round((count1sat+count2sat)*100/(numtrig-lowsnr),2))+'('+str(np.round(count1sat*100/numtrig+count2sat*100/numtrig,2))+ ')'
p1, p2, p3
print(p1+'\t'+p2+'\t'+p3)

24172	163.57 (10.99)	24246	164.07 (11.02)	33188	224.58(15.09)


In [231]:
numtrig = 0
for i in range(len(lumino)):
    if lumino[i]>0:numtrig = numtrig + 1
line1=[] #DD
line2=[] #LIGO
line3=[] #DD + LIGO
line4=[] #D : chosen without any reason as the first among these two (ephem = astrosat1)
line5=[] #D + LIGO
line6=[] #DD detections which LIGO didn't see
line7=[] #D detections which LIGO didn't see
print(numtrig)
# senssattemp = 1.4*10**(-7)
# senssattemp = 7.0*10**(-8)
for i in range(12):
    snr=i
    print('\nSNR =', snr)
    boolvis = []
    countsaa1, countsens1, countearth1, countvis1= 0,0,0,0
    countsaa2, countsens2, countearth2, countvis2 = 0,0,0,0
    count1sat,count1sat = 0,0
    count2sat,count2sat = 0,0
    count0sat,count0sat = 0,0
    vis1, vis2 = 0,0
    satd0, satd1, satd2 = 0,0,0
    dnol, ddnol0, ddnol1,ddnol2 = 0,0,0,0
    lowsnr = 0
    for i in range(len(homotime)):
        if i%10000==0:
            print(int(i*100/len(homotime)),'%',end='\r', flush=True)
        if snrLHVKI[i]<snr:
                lowsnr=lowsnr + 1
        if lumino[i]>0:
            satyes, yes, var3 = 0,0,0
            if is_saa(sublat1[i],sublong1[i], homotime[i])==True:
                countsaa1=countsaa1 + 1
            if sensitivitydata1[i]>lumino[i]:
                countsens1=countsens1+1
            if is_visible(homotime[i],sublat1[i],sublong1[i],np.deg2rad(90.0) - theta1[i],phi1[i], astrosat1)==False:
                countearth1 = countearth1 +1
            if is_saa(sublat1[i],sublong1[i], homotime[i])==False and snrLHVKI[i]>=snr and sensitivitydata1[i]<=lumino[i] and is_visible(homotime[i],sublat1[i],sublong1[i],np.deg2rad(90.0) - theta1[i],phi1[i], astrosat1)==True:
                countvis1 = countvis1+1
                satyes = satyes + 1
            if is_saa(sublat1[i],sublong1[i], homotime[i])==False and sensitivitydata1[i]<=lumino[i] and is_visible(homotime[i],sublat1[i],sublong1[i],np.deg2rad(90.0) - theta1[i],phi1[i], astrosat1)==True:
                vis1 = vis1+1
                yes = yes + 1
            if is_saa(sublat1[i],sublong1[i], homotime[i])==False and snrLHVKI[i]<snr and sensitivitydata1[i]<=lumino[i] and is_visible(homotime[i],sublat1[i],sublong1[i],np.deg2rad(90.0) - theta1[i],phi1[i], astrosat1)==True:
                var3=var3+1
                dnol=dnol+1
                
            if is_saa(sublat2[i],sublong2[i], homotime[i])==True:
                countsaa2=countsaa2 + 1
            if sensitivitydata2[i]>lumino[i]:
                countsens2=countsens2+1
            if is_visible(homotime[i],sublat2[i],sublong2[i],np.deg2rad(90.0) - theta2[i],phi2[i], astrosat1)==False:
                countearth2 = countearth2 +1
            if is_saa(sublat2[i],sublong2[i], homotime[i])==False and snrLHVKI[i]>=snr and sensitivitydata2[i]<=lumino[i] and is_visible(homotime[i],sublat2[i],sublong2[i],np.deg2rad(90.0) - theta2[i],phi2[i], astrosat2)==True:
                countvis2 = countvis2+1
                satyes = satyes + 1
            if is_saa(sublat2[i],sublong2[i], homotime[i])==False and sensitivitydata2[i]<=lumino[i] and is_visible(homotime[i],sublat2[i],sublong2[i],np.deg2rad(90.0) - theta2[i],phi2[i], astrosat2)==True:
                vis2 = vis2+1
                yes = yes + 1           
            if is_saa(sublat2[i],sublong2[i], homotime[i])==False and snrLHVKI[i]<snr and sensitivitydata2[i]<=lumino[i] and is_visible(homotime[i],sublat2[i],sublong2[i],np.deg2rad(90.0) - theta2[i],phi2[i], astrosat2)==True:
                var3=var3+1

            if satyes==0:
                count0sat=count0sat+1
            if satyes==1:
                count1sat=count1sat+1
            if satyes==2:
                count2sat=count2sat+1
            if yes==0:
                satd0=satd0+1
            if yes==1:
                satd1=satd1+1
            if yes==2:
                satd2=satd2+1
            if var3==0:
                ddnol0=ddnol0+1
            if var3==1:
                ddnol1=ddnol1+1
            if var3==2:
                ddnol2=ddnol2+1
    
    print('***100% Completed***\n',end='\r',flush=True)        
    print('lowsnr', lowsnr,'\t', lowsnr*100/numtrig)
    print('goodsnr', numtrig - lowsnr,'\t', (numtrig - lowsnr)*100/numtrig)
    print('\nDL')
    print('countvis1', countvis1,'\t', countvis1*100/(numtrig-lowsnr))
    print('countvis2', countvis2,'\t', countvis2*100/(numtrig-lowsnr))
    print('oD')
    print('vis1', vis1,'\t', vis1*100/(numtrig))
    print('vis2', vis2,'\t', vis2*100/(numtrig))
    print('\nDL', count0sat, count1sat, count2sat)
    print('oD', satd0, satd1, satd2)
    print((count1sat+count2sat+count0sat)*100/numtrig, '-- should be equal to 100%\n')
    print((count1sat+count2sat), '\t',(count1sat+count2sat)*100/(numtrig-lowsnr) ,'(' + str(count1sat*100/numtrig+count2sat*100/numtrig)+ ')')
    print('Line 1 element(Seen by D-Daksha)                      ', satd1+satd2)
    print('Line 2 element(Seen by Ligo network)                  ', len(homotime)-lowsnr)
    print('Line 3 element(Seen by both D-Daksha & LIGO)          ', count1sat+count2sat)
    print('Line 4 element(Seen by only one Daksha)               ', vis1)
    print('Line 5 element(Seen by both Daksha & LIGO)            ', countvis1)
    print('Line 6 element(LIGO missed which D-Dakhsa detected)   ', ddnol1+ddnol2)
    print('Line 7 element(LIGO missed which Daksha detected)     ', dnol)
    line1.append(satd1+satd2)
    line2.append(len(homotime)-lowsnr)
    line3.append(count1sat+count2sat)
    line4.append(vis1)
    line5.append(countvis1)
    line6.append(ddnol1+ddnol2)
    line7.append(dnol)

220000

SNR = 0
***100% Completed***
lowsnr 0 	 0.0
goodsnr 220000 	 100.0

DL
countvis1 86414 	 39.27909090909091
countvis2 86193 	 39.178636363636365
oD
vis1 86414 	 39.27909090909091
vis2 86193 	 39.178636363636365

DL 88376 90641 40983
oD 88376 90641 40983
100.0 -- should be equal to 100%

131624 	 59.82909090909091 (59.82909090909091)
Line 1 element(Seen by D-Daksha)                       131624
Line 2 element(Seen by Ligo network)                   220000
Line 3 element(Seen by both D-Daksha & LIGO)           131624
Line 4 element(Seen by only one Daksha)                86414
Line 5 element(Seen by both Daksha & LIGO)             86414
Line 6 element(LIGO missed which D-Dakhsa detected)    0
Line 7 element(LIGO missed which Daksha detected)      0

SNR = 1
***100% Completed***
lowsnr 156 	 0.07090909090909091
goodsnr 219844 	 99.9290909090909

DL
countvis1 86364 	 39.28421971943742
countvis2 86151 	 39.18733283601099
oD
vis1 86414 	 39.27909090909091
vis2 86193 	 39.1786363636363

***100% Completed***
lowsnr 205222 	 93.28272727272727
goodsnr 14778 	 6.717272727272727

DL
countvis1 8525 	 57.68710244958722
countvis2 8551 	 57.86303965353905
oD
vis1 86414 	 39.27909090909091
vis2 86193 	 39.178636363636365

DL 208132 6660 5208
oD 88376 90641 40983
100.0 -- should be equal to 100%

11868 	 80.30856678846935 (5.3945454545454545)
Line 1 element(Seen by D-Daksha)                       131624
Line 2 element(Seen by Ligo network)                   14778
Line 3 element(Seen by both D-Daksha & LIGO)           11868
Line 4 element(Seen by only one Daksha)                86414
Line 5 element(Seen by both Daksha & LIGO)             8525
Line 6 element(LIGO missed which D-Dakhsa detected)    119756
Line 7 element(LIGO missed which Daksha detected)      77889


In [170]:
count1satdl+count2satdl, count1satd+count2satd, countvis1d, count1visdl, countvis2d, countvis2dl
len(line3), len(line4), len(line5)

(12, 12, 12)

In [232]:
tab = Table([line2,line1,line3,line4,line5,line6,line7], names=('LIGO','DD','DD+LIGO','D','D+L','DD-LIGO','D-LIGO'))
tab

LIGO,DD,DD+LIGO,D,D+L,DD-LIGO,D-LIGO
int64,int64,int64,int64,int64,int64,int64
220000,131624,131624,86414,86414,0,0
219844,131624,131546,86414,86364,78,50
215225,131624,128920,86414,84704,2704,1710
193553,131624,117437,86414,77472,14187,8942
155093,131624,96498,86414,64218,35126,22196
118344,131624,75680,86414,50934,55944,35480
86218,131624,56957,86414,38714,74667,47700
59060,131624,41068,86414,28366,90556,58048
39675,131624,29032,86414,20360,102592,66054


In [235]:
norm = 1/(line2[8]/100)
lines = []
lines.append(np.array(line1))
lines.append(np.array(line2))
lines.append(np.array(line3))
lines.append(np.array(line4))
lines.append(np.array(line5))
lines.append(np.array(line6))
lines.append(np.array(line7))
lines[0] = lines[0]*norm
lines[1] = lines[1]*norm
lines[2] = lines[2]*norm
lines[3] = lines[3]*norm
lines[4] = lines[4]*norm
lines[5] = lines[5]*norm
lines[6] = lines[6]*norm
plt.plot(np.arange(0,12),lines[0], marker='o', label='D-Daksha total detections')
plt.plot(np.arange(0,12),lines[1], marker='^', label='LIGO detections')
plt.plot(np.arange(0,12),lines[2], marker='+', label='D-Daksha detections among LIGO detections')
plt.plot(np.arange(0,12),lines[3], marker='v', label='Single Daksha total detections')
plt.plot(np.arange(0,12),lines[4], marker='x', label='Single Daksha detections among LIGO detections')
plt.plot(np.arange(0,12),lines[5], marker='x', label='LIGO missed detections which D-Daksha detected')
plt.plot(np.arange(0,12),lines[6], marker='x', label='LIGO missed detections which Daksha detected')

plt.xlabel('SNR threshold', size=12)
plt.ylabel('Number of Detections', size='12')
plt.title('Comparison of Double Daksha, Daksha and LIGO detections (LHVKI)', size=20)
plt.legend()
plt.grid()

In [219]:
for i in range(len(line4)):
    print(line4[i] - line5[i])
#     print(line1[i])

0
47
1678
8805
21954
35307
47404
57681
65700
71118
74848
77470
